In [6]:

import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict

filename = 'osm/sample.osm'


In [3]:
def count_tags(filename):
    osm_file = open(filename, "r")
    print ('opened_file')
    tagsdict = defaultdict (lambda: 0)
    #for event, elem in ET.iterparse(osm_file, events=('start', )):
    #    tagsdict[elem.tag]=tagsdict[elem.tag]+1
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end':
            tagsdict[elem.tag]=tagsdict[elem.tag]+1
            root.clear()    
 
    return dict(tagsdict)

tags = count_tags(filename)
pprint.pprint(tags)    


opened_file
{'member': 31,
 'nd': 11796,
 'node': 9413,
 'osm': 1,
 'relation': 8,
 'tag': 8828,
 'way': 1539}


In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Get a list of all the tags and their count
"""

def count_tags(filename):
    osm_file = open(filename, "r")
    tagsdict = defaultdict (lambda: 0)
    for event, elem in ET.iterparse(osm_file, events=('start', )):
        tagsdict[elem.tag]=tagsdict[elem.tag]+1
    return dict(tagsdict)

tags = count_tags(filename)
pprint.pprint(tags)    


{'member': 31,
 'nd': 11796,
 'node': 9413,
 'osm': 1,
 'relation': 8,
 'tag': 8828,
 'way': 1539}


In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
check that the tag keys are valid chars
"""
import re
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}


osm_file = open(filename, "r")
context = iter(ET.iterparse(osm_file, events=('start', 'end')))
_, root = next(context)
for event, element in context:
    if event == 'end':
        if element.tag == "tag":
            key = element.attrib['k']
            whichCase = "other"
            if (lower.match(key)): whichCase = "lower"
            elif (lower_colon.match(key)): whichCase = "lower_colon"
            elif (problemchars.search(key)): whichCase = "problemchars"
            #print key, whichCase
            keys [whichCase] = keys [whichCase] + 1
    root.clear()
    
pprint.pprint(keys)

{'lower': 3452, 'lower_colon': 5243, 'other': 106, 'problemchars': 27}


In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
get the unique users that have modified the map
"""

def get_attrib(element, key):
    if ( key in element.attrib ):
        return element.attrib[key]
    return None

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        user  = get_attrib(element, 'user')
        if user != None: users.add(user)
    return users

users = process_map(filename)
pprint.pprint(users)


set(['1248',
     '1HealthPlz',
     '3yoda',
     '42429',
     'ALE!',
     'Aaron Lidman',
     'Abeer2015',
     'Alanna Smith',
     'AlexTheTux',
     'Andre Engels',
     'Andre68',
     'Anthony Moffa',
     'Arboreal',
     'Arcadium',
     'BMM994',
     u'BlaueBl\xfcte',
     'Bow717588',
     'Brett Camper',
     'Bri Hendriksen',
     'CS9229',
     'CaptainCrunch',
     'Cato_d_Ae',
     'Chris Parker',
     'ChrisZontine',
     'Chrisi_LE',
     'Christian70',
     'Christoph Lotz',
     'CitymapperHQ',
     'ColinReilly',
     'Constable',
     'CoreyFarwell',
     'Craig Williams',
     'Dami_Tn',
     'DanX',
     'DaveHansenTiger',
     'DawsZed',
     'DeVietor',
     'DennisL',
     'DerRote',
     'Derick Rethans',
     'Dirk Lohse',
     'Easky30',
     'Edward',
     'Eliyak',
     'ElliottPlack',
     'Eric Godwin',
     'Ericles',
     'Extramiler',
     'FitmanNJ',
     'Fluous',
     'GISJoe',
     'GaryS5',
     'Geofreund1',
     'Geogast',
     'GinosNY',

In [5]:
"""
Audit Street Types
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
first_word_re = re.compile(r'^\w+', re.IGNORECASE)
old_expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Terrace", "Loop", "Highway", "Course", "Circle", "Way", "Crescent", "Walk"]
expected_first = ["Avenue"]
mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Rd." : "Road",
            "Rd" : "Road"
            }
extra_suffix = ["North", "East", "West", "South"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            #print 'Found', street_type
            if street_type_first(street_name): return
            else: street_types[street_type].add(street_name)
            
def street_type_first(street_name):
    #print 'Check first', street_name
    m = first_word_re.search(street_name)
    if m:
        if m.group() in expected_first:
            #print 'Found Street Type first for', street_name
            return True
    return False
            
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit_street_types_in_file(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    #print tag.attrib['k'], tag.attrib['v']
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        
        #print "Checking ", name
        if street_type in expected: pass
        elif street_type in mapping: 
            print "Fixing name ", name
            name = street_type_re.sub(mapping[street_type], name)
            #print "Fixed name ", name
        else:
            processed_suffix = process_suffix(name, mapping)
            if processed_suffix != None: return processed_suffix
        
        return name     

def process_suffix (name, mapping):
    split_name = name.split(' ')
    if len(split_name) > 2 and split_name[-1] in extra_suffix:
        print "Has a valid suffix ", name
        return update_name(name.rsplit(' ', 1)[0], mapping) + ' ' + split_name[-1] 
    return None

unexpected_st_types = audit_street_types_in_file(filename)
pprint.pprint(dict(unexpected_st_types))

for st_type, ways in unexpected_st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name


{'Bowery': set(['Bowery']),
 'Broadway': set(['Broadway']),
 'East': set(['Aqueduct Avenue East',
              'Van Wyck Expressway Service Road East']),
 'Green': set(['Dover Green']),
 'Island': set(['Randalls Island']),
 'North': set(['Mayfair Drive North']),
 'Rd': set(['Knightsbridge Rd']),
 'South': set(['Andrews Ave South',
               'Horace Harding Expressway Service Road South']),
 'West': set(['Central Park West',
              'Clearview Expressway Service Road West',
              'Prospect Park West',
              'Van Wyck Expressway Service Road West'])}
Has a valid suffix  Mayfair Drive North
Mayfair Drive North => Mayfair Drive North
Has a valid suffix  Central Park West
Central Park West => Central Park West
Has a valid suffix  Van Wyck Expressway Service Road West
Van Wyck Expressway Service Road West => Van Wyck Expressway Service Road West
Has a valid suffix  Clearview Expressway Service Road West
Clearview Expressway Service Road West => Clearview Expresswa

In [52]:
import codecs
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
replace_tags = {'addr':'address'}
#has_value_and_children = ['hgv', 'name', 'building', 'railway', 'lanes', 'maxspeed', 'source']
has_value_and_children = []

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [58]:
def process_tag (name, value, tags, found_complex_tag):
    #tags[name]=value
    #if (name.startswith('hgv')): print name, value
    try:
        if (name!= None and len(name)>0 and name.strip()!=':' and 
            name.strip()!='' and not problemchars.search(name)):
            
            levels = name.split(':')
            
            #extract the top level key and convert it as needed
            top = levels[0]
            top = replace_tags.get(top,top)
            
            #assign value if it's a top level attribute
            if len(levels) == 1: 
                if top not in tags: tags[top] = value
                else: 
                    print top, value, ' other value already found as ', tags[top]
                
            #add to address dict if there are two levels
            elif top=='address' and len(levels) == 2:
                if 'address' not in tags: tags['address']={}                    
                if not isinstance(tags['address'],dict): 
                    print 'address:', name, value, ' overwriting simple value already found as ', tags['address']
                    tags['address']={}
                process_tag (levels[1],value,tags['address'], found_complex_tag)
                
            #add to a dict if there are multiple levels
            elif top!='address' and len(levels) > 1:
                if top in has_value_and_children: top = top + "_data"
                elif top in tags and not isinstance(tags[top],dict): 
                    #a root value was already found for this tag.
                    print name, value, ' unexpected - simple value already found as ', top, tags[top]
                    found_complex_tag.add(top)
                    return
                if top not in tags: tags[top]={}                    
                process_tag (name.split(':', 1)[1], value, tags[top], found_complex_tag)
                
    except Exception, e:
        print "Failed: %s" % e
        print 'Exception', name, value
        pprint.pprint(tags)
        return

In [59]:
def process_attributes(element):
    attributes = {}
    created = {}
    #address = {}
    pos = [None,None]

    for attribute in element.attrib:
        value = element.attrib[attribute]
        #print attribute

        if attribute in CREATED:
            created[attribute] = value

        elif attribute == 'lat': 
            try:
                pos [0] = float(value)
            except ValueError:
                print "Not a float"                
        elif attribute == 'lon':
            try:
                pos [1] = float(value)
            except ValueError:
                print "Not a float"

        else: attributes[attribute] = value    

    attributes['created'] = created
    if None not in pos: attributes['pos'] = pos
        
    return attributes    

def process_refs(element):
    node_refs = []
    for ref in element.iter("nd"):
        node_refs.append(ref.attrib['ref'])
    if len(node_refs) > 0 : return {"node_refs":node_refs}
    return {}

def process_tags(element, found_complex_tag):
    tags = {}
    for tag in element.iter("tag"):
        process_tag(tag.attrib['k'],tag.attrib['v'], tags, found_complex_tag)
    return tags

In [60]:
street_tags = ['street']
def fix_street(tags):
    if 'address' in tags:
        addr = tags['address']
        for key in addr:
            if key in street_tags:
                #print 'checking', addr[key]
                addr[key] = update_name(addr[key], mapping)

In [61]:
def shape_element(element, found_complex_tag):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node['type']=element.tag        
        refs = process_refs(element)
        node.update(refs)
        attrs = process_attributes(element)    
        node.update(attrs)
        tags = process_tags(element, found_complex_tag)
        fix_street(tags)
        node.update(tags)
              
        #pprint.pprint( node )
        return node
    else:
        return None

In [62]:
"""
Parse an OSM XML file and call shape_element for each entry,
the shaped element is written to a file in json format
"""
def check_has_value_and_children(file_in, pretty = False):
    #data = []
    found_complex_tag = set()
    osm_file = open(filename, "r")
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end':
            el = shape_element(elem, found_complex_tag)
        root.clear()
    return found_complex_tag
print check_has_value_and_children(filename, False)

Has a valid suffix  Prospect Park West
Has a valid suffix  Clearview Expressway Service Road West
Has a valid suffix  Andrews Ave South
Fixing name  Andrews Ave
maxspeed:hgv 50 mph  unexpected - simple value already found as  maxspeed 55 mph
maxspeed:goods 50 mph  unexpected - simple value already found as  maxspeed 55 mph
hgv:national_network yes  unexpected - simple value already found as  hgv designated
hgv:state_network yes  unexpected - simple value already found as  hgv designated
Fixing name  Knightsbridge Rd
hgv:state_network yes  unexpected - simple value already found as  hgv designated
name:en Bakery  unexpected - simple value already found as  name Bakery
Has a valid suffix  Mayfair Drive North
Has a valid suffix  Horace Harding Expressway Service Road South
building:levels 20  unexpected - simple value already found as  building apartments
Has a valid suffix  Central Park West
Has a valid suffix  Aqueduct Avenue East
Has a valid suffix  Van Wyck Expressway Service Road Wes

In [32]:
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

x = process_map(filename, False)

TypeError: shape_element() takes exactly 2 arguments (1 given)

In [16]:
import codecs
import json
"""
Parse an OSM XML file and call shape_element for each entry,
the shaped element is written to a file in json format
"""
def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        osm_file = open(filename, "r")
        context = iter(ET.iterparse(osm_file, events=('start', 'end')))
        _, root = next(context)
        for event, elem in context:
            if event == 'end':
                el = shape_element(elem)
                if el:
                    data.append(el)
                    if pretty:
                        fo.write(json.dumps(el, indent=2)+"\n")
                    else:
                        fo.write(json.dumps(el) + "\n")
            root.clear()
    return data

x = process_map(filename, False)

Has a valid suffix  Prospect Park West
Has a valid suffix  Clearview Expressway Service Road West
Has a valid suffix  Andrews Ave South
Fixing name  Andrews Ave
Fixing name  Knightsbridge Rd
Has a valid suffix  Mayfair Drive North
Has a valid suffix  Horace Harding Expressway Service Road South
Has a valid suffix  Central Park West
Has a valid suffix  Aqueduct Avenue East
Has a valid suffix  Van Wyck Expressway Service Road West
Has a valid suffix  Van Wyck Expressway Service Road East


In [12]:
import json

def extract_tags(node,parent,tags):
    for key in node:
        #print key, node[key]
        if isinstance (node[key],dict): 
            #print "found dict for ", key
            extract_tags(node[key] , parent + key + ".", tags)
        else:
            if key not in tags: tags[key] = set()
            tags[key].add( parent+key )

def check_for_unique_tags(jsonFile):
    f = open(jsonFile)
    tags = defaultdict(set)
    for line in iter(f):
        #print line
        node = json.loads(line)
        extract_tags(node, '', tags)
    f.close()
    #pprint.pprint (dict(tags))
    multis = {k: v for k, v in (dict(tags)).items() if len(tags[k])>1}
    pprint.pprint (multis)
    
check_for_unique_tags(filename + '.json')


{u'both_ways': set([u'lanes_data.both_ways', u'turn.lanes_data.both_ways']),
 u'colour': set([u'building_data.colour', u'roof.colour']),
 u'forward': set([u'lanes_data.forward', u'turn.lanes_data.forward']),
 u'goods': set([u'goods', u'maxspeed_data.goods']),
 u'height': set([u'height', u'roof.height']),
 u'hgv': set([u'hgv', u'maxspeed_data.hgv', u'source_data.hgv']),
 u'id': set([u'gnis.id', u'id']),
 u'import_uuid': set([u'gnis.import_uuid', u'import_uuid']),
 u'lanes': set([u'lanes', u'note.lanes', u'turn.lanes']),
 u'material': set([u'building_data.material', u'material', u'roof.material']),
 u'national_network': set([u'hgv_data.national_network',
                           u'source_data.hgv_data.national_network']),
 u'old_ref': set([u'old_ref', u'source_data.old_ref']),
 u'reviewed': set([u'gnis.reviewed', u'tiger.reviewed']),
 u'source': set([u'generator.source', u'source', u'tiger.source']),
 u'state_network': set([u'hgv_data.state_network',
                        u'source_da

In [13]:
def isfloat(value):
  try:
    float(value)
    return True
  except:
    return False

def isint(value):
  try:
    int(value)
    return True
  except:
    return False

def extract_data_types(node,parent,tags):
    for key in node:
        location = parent + key
        #print key, node[key]
        if isinstance (node[key],dict): 
            #print "found dict for ", key
            extract_data_types(node[key] , location + ".", tags)
        else:
            if location not in tags: tags[location] = set()
            tags[location].add( get_data_type(node[key]) )

def get_data_type(value):
    if value == "" or value == "NULL":
        return type(None)
    elif type(value) is list:
        return type([])
    elif isint(value):
        return type(int())
    elif isfloat(value):
        return type(float())
    else:
        return type(str())

def audit_data_types_in_file(jsonFile):
    f = open(jsonFile)
    tags = defaultdict(set)
    for line in iter(f):
        #print line
        node = json.loads(line)
        extract_data_types(node, '', tags)
    f.close()
    return dict(tags)
    
data_types = audit_data_types_in_file(filename + '.json')

In [14]:

pprint.pprint(data_types)

{u'Bergen_County_database_ref': set([<type 'int'>]),
 u'FIXME': set([<type 'str'>]),
 u'HFCS': set([<type 'str'>]),
 u'NHS': set([<type 'str'>]),
 u'NJDOT_SRI': set([<type 'int'>, <type 'str'>]),
 u'access': set([<type 'str'>]),
 u'address.city': set([<type 'str'>]),
 u'address.country': set([<type 'str'>]),
 u'address.housenumber': set([<type 'int'>, <type 'str'>]),
 u'address.postcode': set([<type 'int'>]),
 u'address.state': set([<type 'str'>]),
 u'address.street': set([<type 'str'>]),
 u'admin_level': set([<type 'int'>]),
 u'alt_name': set([<type 'str'>]),
 u'amenity': set([<type 'str'>]),
 u'area': set([<type 'str'>]),
 u'backrest': set([<type 'str'>]),
 u'barrier': set([<type 'str'>]),
 u'bench': set([<type 'str'>]),
 u'bicycle': set([<type 'str'>]),
 u'boundary': set([<type 'str'>]),
 u'bridge': set([<type 'str'>]),
 u'building': set([<type 'str'>]),
 u'building_data.colour': set([<type 'str'>]),
 u'building_data.levels': set([<type 'int'>]),
 u'building_data.material': set([<ty

In [15]:
def pick_data_type(data_types):
    pick_type = {}
    for key in data_types:
        if  type([]) in data_types[key]:
            pick_type [key] = type([])
        elif  type(str()) in data_types[key]:
            pick_type [key] = type(str())
        elif  type(float()) in data_types[key]:
            pick_type [key] = type(float())
        elif  type(int()) in data_types[key]:
            pick_type [key] = type(int())
        else:
            pick_type [key] =  type(str())
    return pick_type

data_types = audit_data_types_in_file(filename + '.json')
save_as_data_type = pick_data_type(data_types)
#pprint.pprint (save_as_data_type)

ints = [ key for (key, value) in save_as_data_type.items() if value == type(int()) and not 'zip' in key]
print ints
#not_ints = [ key for key in ints if key == type(int()) ]

[u'gauge', u'voltage', u'ele', u'roof.height', u'gnis.id', u'Bergen_County_database_ref', u'circuits', u'created.uid', u'gnis.feature_id', u'frequency', u'capacity', u'gnis.County_num', u'lanes_data.both_ways', u'gnis.ST_num', u'width', u'start_date', u'admin_level', u'lanes', u'gnis.state_id', u'layer', u'created.version', u'nycdoitt.bin', u'gnis.county_id', u'cables', u'id', u'lanes_data.forward', u'building_data.levels', u'created.changeset', u'railway_data.track_ref', u'address.postcode', u'min_height']


In [16]:
def get_db():
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    # 'examples' here is the database name. It will be created if it does not exist.
    db = client.osm
    return db

db = get_db()
db.sample.find_one()

#db.test.update({field:"string"}, {$set:{field:23}})

{u'_id': ObjectId('581c649fd123cce95cd02898'),
 u'created': {u'changeset': u'41015803',
  u'timestamp': u'2016-07-25T17:17:46Z',
  u'uid': u'326503',
  u'user': u'wambag',
  u'version': u'4'},
 u'id': u'26769789',
 u'pos': [40.6995927, -74.1868914],
 u'type': u'node'}

In [17]:
db.sample.find_one({"address.postcode" : { "$exists" : True } })


{u'_id': ObjectId('581c649fd123cce95cd02f40'),
 u'address': {u'city': u'Brooklyn',
  u'housenumber': u'14',
  u'postcode': u'11201',
  u'street': u'College Place'},
 u'created': {u'changeset': u'16297856',
  u'timestamp': u'2013-05-26T17:49:08Z',
  u'uid': u'904598',
  u'user': u'SarahHaskins',
  u'version': u'1'},
 u'id': u'2321029592',
 u'pos': [40.6962445, -73.9943807],
 u'type': u'node'}

In [18]:
def make_zip_pipeline():
    # complete the aggregation pipeline
    
    #match brasilia time zone and number of tweets > 100
    #project to _id, followers, screen_name, tweets
    #sort by followers descenging
    
    pipeline = [ 
        { "$match": { "address.postcode" : { "$exists" : True } } },
        { "$group" : { "_id": "$address.postcode", "count" : {"$sum" : 1 }}},
        { "$sort": {"_id":1 }} ]
        #{ "$sort": {"address.postcode":-1 }},
        #{ "$limit" : 1 } ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.sample.aggregate(pipeline)]


db = get_db()
pipeline = make_zip_pipeline()
result = aggregate(db, pipeline)
#import pprint
#pprint.pprint(result)
print result

[{u'count': 2, u'_id': u'08816'}, {u'count': 1, u'_id': u'08854'}, {u'count': 3, u'_id': u'10001'}, {u'count': 2, u'_id': u'10002'}, {u'count': 3, u'_id': u'10003'}, {u'count': 1, u'_id': u'10007'}, {u'count': 2, u'_id': u'10009'}, {u'count': 2, u'_id': u'10011'}, {u'count': 2, u'_id': u'10012'}, {u'count': 2, u'_id': u'10013'}, {u'count': 2, u'_id': u'10014'}, {u'count': 2, u'_id': u'10016'}, {u'count': 1, u'_id': u'10018'}, {u'count': 3, u'_id': u'10019'}, {u'count': 2, u'_id': u'10021'}, {u'count': 1, u'_id': u'10022'}, {u'count': 3, u'_id': u'10023'}, {u'count': 3, u'_id': u'10024'}, {u'count': 1, u'_id': u'10025'}, {u'count': 2, u'_id': u'10027'}, {u'count': 1, u'_id': u'10028'}, {u'count': 3, u'_id': u'10029'}, {u'count': 1, u'_id': u'10030'}, {u'count': 1, u'_id': u'10031'}, {u'count': 2, u'_id': u'10032'}, {u'count': 1, u'_id': u'10033'}, {u'count': 1, u'_id': u'10034'}, {u'count': 4, u'_id': u'10035'}, {u'count': 1, u'_id': u'10039'}, {u'count': 3, u'_id': u'10065'}, {u'count'

In [19]:
res = db.sample.find({"address.postcode" : '08816'})
for rec in res:
    pprint.pprint (rec)

{u'_id': ObjectId('581c64a1d123cce95cd052a8'),
 u'address': {u'city': u'East Brunswick',
              u'country': u'US',
              u'housenumber': u'304',
              u'postcode': u'08816',
              u'state': u'NJ',
              u'street': u'Palombi Court'},
 u'building': u'terrace',
 u'building_data': {u'levels': u'2'},
 u'created': {u'changeset': u'37964404',
              u'timestamp': u'2016-03-20T21:29:09Z',
              u'uid': u'95246',
              u'user': u'holub',
              u'version': u'2'},
 u'id': u'318881070',
 u'node_refs': [u'3252748371',
                u'3252748372',
                u'3252748373',
                u'3252748374',
                u'3252748371'],
 u'pos': [None, None],
 u'type': u'way'}
{u'_id': ObjectId('581c64a1d123cce95cd052f4'),
 u'address': {u'city': u'East Brunswick',
              u'country': u'US',
              u'housenumber': u'8',
              u'postcode': u'08816',
              u'state': u'NJ',
              u'street': u'

In [20]:
pprint.pprint(db.sample.find_one({"address.postcode" : '08854'}))

{u'_id': ObjectId('581c64a1d123cce95cd04e21'),
 u'address': {u'housenumber': u'53',
              u'postcode': u'08854',
              u'street': u'Knightsbridge Road'},
 u'alt_name': u'www.inetsoft.com',
 u'building': u'office',
 u'created': {u'changeset': u'15362374',
              u'timestamp': u'2013-03-14T13:27:48Z',
              u'uid': u'1267660',
              u'user': u'InetSoftTech',
              u'version': u'3'},
 u'id': u'204460662',
 u'name': u'InetSoft Technology',
 u'node_refs': [u'2144693655',
                u'2144693182',
                u'2144693242',
                u'2144693723',
                u'2144693655'],
 u'pos': [None, None],
 u'source': u'Bing',
 u'type': u'way'}


In [21]:
wrong_length = [ rec['_id'] for rec in result if len(rec['_id']) != 5]

##for rec in result:
##    zip = rec['_id']
##    wrong_length.append(zip)

print wrong_length

[u'115422']


In [22]:
pprint.pprint(db.sample.find_one({"address.postcode" : '115422'}))

{u'_id': ObjectId('581c64a1d123cce95cd0534e'),
 u'address': {u'city': u'Glen Cove',
              u'country': u'US',
              u'housenumber': u'17',
              u'postcode': u'115422',
              u'state': u'NY',
              u'street': u'Ford Street'},
 u'building': u'house',
 u'created': {u'changeset': u'42162760',
              u'timestamp': u'2016-09-15T02:49:53Z',
              u'uid': u'19488',
              u'user': u'miluethi',
              u'version': u'1'},
 u'id': u'442516108',
 u'node_refs': [u'4401968681',
                u'4401968682',
                u'4401968686',
                u'4401968685',
                u'4401968681'],
 u'pos': [None, None],
 u'type': u'way'}


In [23]:
res = db.sample.find({"pos" : None})
print res.count()
res = db.sample.find()
print res.count()

1539
10952


In [24]:
def make_distinct_with_count_pipeline( by_field ):    
    pipeline = [ 
        { "$match": { by_field : { "$exists" : True } } },
        { "$group" : { "_id": "$" + by_field , "count" : {"$sum" : 1 }}},
        { "$sort": {"count":-1 }} ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.sample.aggregate(pipeline)]

def distinct_with_count(field_name):
    db = get_db()
    pipeline = make_distinct_with_count_pipeline(field_name)
    result = aggregate(db, pipeline)
    return result

#import pprint
#pprint.pprint(result)
result = distinct_with_count('address.state')
print result

[{u'count': 8, u'_id': u'NY'}, {u'count': 2, u'_id': u'NJ'}]


In [25]:
def make_countdistinct_pipeline( by_field ):    
    pipeline = [ 
        { "$match": { by_field : { "$exists" : True } } },
        { "$group" : { "_id": "$" + by_field }},
        { "$group": { "_id": by_field, "count" : {"$sum" : 1 } } } ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.sample.aggregate(pipeline)]

def count_distinct(field_name):
    db = get_db()
    pipeline = make_countdistinct_pipeline(field_name)
    result = aggregate(db, pipeline)
    return result[0]

#import pprint
#pprint.pprint(result)
result = count_distinct('address.state')
print result

{u'count': 2, u'_id': u'address.state'}


In [26]:
res = db.sample.find({"address.state" : {"$exists": True , "$ne": 'NY'}})
#print res
for rec in res:
    pprint.pprint (rec)

{u'_id': ObjectId('581c64a1d123cce95cd052a8'),
 u'address': {u'city': u'East Brunswick',
              u'country': u'US',
              u'housenumber': u'304',
              u'postcode': u'08816',
              u'state': u'NJ',
              u'street': u'Palombi Court'},
 u'building': u'terrace',
 u'building_data': {u'levels': u'2'},
 u'created': {u'changeset': u'37964404',
              u'timestamp': u'2016-03-20T21:29:09Z',
              u'uid': u'95246',
              u'user': u'holub',
              u'version': u'2'},
 u'id': u'318881070',
 u'node_refs': [u'3252748371',
                u'3252748372',
                u'3252748373',
                u'3252748374',
                u'3252748371'],
 u'pos': [None, None],
 u'type': u'way'}
{u'_id': ObjectId('581c64a1d123cce95cd052f4'),
 u'address': {u'city': u'East Brunswick',
              u'country': u'US',
              u'housenumber': u'8',
              u'postcode': u'08816',
              u'state': u'NJ',
              u'street': u'

In [27]:
db = get_db()
pipeline = make_countdistinct_pipeline('address.city')
result = aggregate(db, pipeline)
#import pprint
pprint.pprint(result)

[{u'_id': u'address.city', u'count': 6}]


In [28]:
res = db.sample.find({"$and":[{"tiger.name_base" : {"$exists": True }}, 
                              {"address.street": {"$exists": True}} ] })
print res.count()
#for rec in res:
#    pprint.pprint (rec)

res = db.sample.find_one({"tiger.name_base": {"$exists": True}})
pprint.pprint(res)

matchy = db.sample.find({"address.street": {"$regex" : "82"} })
print matchy.count()
for rec in matchy: pprint.pprint (rec)

0
{u'_id': ObjectId('581c64a1d123cce95cd04d5d'),
 u'created': {u'changeset': u'28749348',
              u'timestamp': u'2015-02-10T12:37:06Z',
              u'uid': u'51722',
              u'user': u'Chris Parker',
              u'version': u'7'},
 u'highway': u'residential',
 u'id': u'5670797',
 u'name': u'East 82nd Street',
 u'node_refs': [u'42439389',
                u'3341858523',
                u'3341917065',
                u'42439392',
                u'3341917066',
                u'3341912573',
                u'596776127',
                u'42439395',
                u'3341912575',
                u'42439399',
                u'42439403',
                u'42439406',
                u'42439409',
                u'42428027',
                u'42439416'],
 u'oneway': u'yes',
 u'pos': [None, None],
 u'tiger': {u'cfcc': u'A41',
            u'county': u'New York, NY',
            u'name_base': u'82nd',
            u'name_direction_prefix': u'E',
            u'name_type': u'St',
 

In [29]:
def cross_check_ways ():
    no_match = {}
    maybe_match = {}
    res = db.sample.find({"tiger.name_base": {"$exists": True}})
    print "tiger #", res.count()
    for rec in res:
        name = rec["tiger"]["name_base"]
        matchy = db.sample.find({"address.street": {"$regex" : name} })
        if matchy.count()>0: 
            if name not in maybe_match: 
                maybe_match[name] = [a for a in matchy]
            maybe_match[name].insert(0,rec)
        else: no_match[name] = rec
            
    print "maybe match #", len(maybe_match)
    print "no match #", len(no_match)
    return maybe_match, no_match

(maybe_match, no_match) = cross_check_ways()
#pprint.pprint(maybe_match[maybe_match.keys()[0]])


tiger # 153
maybe match # 48
no match # 101


In [30]:
for street in maybe_match.keys() [:2]:
    print '*****  ' , street, '  *****'
    for rec in maybe_match[street]:
        try: 
            x = rec['address']['street'] + ' ' + '(address)'
            print x
        except: pass
        try:
            x = rec['tiger']['name_direction_prefix']+ ' ' + rec['tiger']['name_base']+ ' ' +  rec['tiger']['name_type']+ ' ' +  '(tiger)'
            print x
        except: pass
        try:
            x = rec['tiger']['name_base']+ ' ' +  rec['tiger']['name_type']+ ' ' +  '(tiger)'
            print x
        except: pass
    
    

*****   82nd   *****
E 82nd St (tiger)
82nd St (tiger)
82nd Street (address)
82nd Drive (address)
West 82nd Street (address)
*****   56th   *****
56th Ave (tiger)
East 156th Street (address)
256th Street (address)
156th Street (address)


In [31]:
size = db.sample.count()
print size

10952


In [32]:
types = count_distinct("type")
pprint.pprint(types)

{u'_id': u'type', u'count': 2}


In [33]:
def print_count_distincts(fields):
    for field_name in fields:
        #pprint.pprint(get_top(field_name))
        count_dist = count_distinct(field_name)
        print "Distinct ", field_name, count_dist['_id'], ' - count', count_dist['count']

get_count_distincts_fields = ['created.user', 'address.postcode', 'address.city', 'amenity','source']
print_count_distincts(get_count_distincts_fields)

Distinct  created.user created.user  - count 377
Distinct  address.postcode address.postcode  - count 169
Distinct  address.city address.city  - count 6
Distinct  amenity amenity  - count 14
Distinct  source source  - count 13


In [34]:
def make_get_top_pipeline( by_field ):    
    pipeline = [ 
        { "$match": { by_field : { "$exists" : True } } },
        { "$group" : { "_id": "$" + by_field , "count" : {"$sum" : 1 }}},
        { "$sort": { "count" :-1 }},
        { "$limit" : 1 } ]
    return pipeline

def get_top(field_name):
    db = get_db()
    pipeline = make_get_top_pipeline(field_name)
    result = aggregate(db, pipeline)
    return result[0]

def print_tops(fields):
    for field_name in fields:
        #pprint.pprint(get_top(field_name))
        top = get_top(field_name)
        print "Top", field_name, top['_id'], ' - count', top['count']

#import pprint
#pprint.pprint(result)
get_top_fields = ['created.user', 'address.postcode', 'address.city', 'amenity','source']
print_tops(get_top_fields)


#([{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$sort":{"count":­1}}, {"$limit":1}])

Top created.user Rub21_nycbuildings  - count 4923
Top address.postcode 11234  - count 24
Top address.city New York  - count 4
Top amenity parking  - count 10
Top source Bing  - count 6


In [35]:
distinct_with_count("cuisine")

[{u'_id': u'french', u'count': 1},
 {u'_id': u'japanese', u'count': 1},
 {u'_id': u'burger', u'count': 1}]

In [36]:
distinct_with_count("source")

[{u'_id': u'Bing', u'count': 6},
 {u'_id': u'NJ2002LULC', u'count': 5},
 {u'_id': {u'hgv_data': {u'state_network': u'NJDOT http://www.state.nj.us/transportation/about/rules/pdf/chapter32truckaccess.pdf'}},
  u'count': 5},
 {u'_id': u'USGS Geonames', u'count': 3},
 {u'_id': u'TIGER/Line\xae 2008 Place Shapefiles (http://www.census.gov/geo/www/tiger/)',
  u'count': 2},
 {u'_id': {u'hgv': u'Rules of the City of New York Title 34 \xa74-13 http://24.97.137.100/nyc/rcny/Title34_4-13.asp'},
  u'count': 2},
 {u'_id': {u'hgv_data': {u'national_network': u'NJDOT http://www.state.nj.us/transportation/about/rules/pdf/chapter32truckaccess.pdf'}},
  u'count': 2},
 {u'_id': u'BingSat, site review in August 2016 and mapillary imagery',
  u'count': 1},
 {u'_id': {u'old_ref': u'1947 and 1951 maps http://mapmaker.rutgers.edu/MIDDLESEX_COUNTY/MiddlesexCounty.html'},
  u'count': 1},
 {u'_id': u'Local Knowladge', u'count': 1},
 {u'_id': u'bing', u'count': 1},
 {u'_id': u'survey', u'count': 1},
 {u'_id': u't

In [37]:
get_distincts = ["cuisine", "source", "leisure", "office", "service", "shelter", "shop", "sport", "gnis.feature_type"]

for field_name in get_distincts:
    print "Distinct", field_name
    pprint.pprint (distinct_with_count(field_name))
    print " "


Distinct cuisine
[{u'_id': u'french', u'count': 1},
 {u'_id': u'japanese', u'count': 1},
 {u'_id': u'burger', u'count': 1}]
 
Distinct source
[{u'_id': u'Bing', u'count': 6},
 {u'_id': u'NJ2002LULC', u'count': 5},
 {u'_id': {u'hgv_data': {u'state_network': u'NJDOT http://www.state.nj.us/transportation/about/rules/pdf/chapter32truckaccess.pdf'}},
  u'count': 5},
 {u'_id': u'USGS Geonames', u'count': 3},
 {u'_id': u'TIGER/Line\xae 2008 Place Shapefiles (http://www.census.gov/geo/www/tiger/)',
  u'count': 2},
 {u'_id': {u'hgv': u'Rules of the City of New York Title 34 \xa74-13 http://24.97.137.100/nyc/rcny/Title34_4-13.asp'},
  u'count': 2},
 {u'_id': {u'hgv_data': {u'national_network': u'NJDOT http://www.state.nj.us/transportation/about/rules/pdf/chapter32truckaccess.pdf'}},
  u'count': 2},
 {u'_id': u'BingSat, site review in August 2016 and mapillary imagery',
  u'count': 1},
 {u'_id': {u'old_ref': u'1947 and 1951 maps http://mapmaker.rutgers.edu/MIDDLESEX_COUNTY/MiddlesexCounty.html'},